In [3]:
import azureml.dataprep as dprep

In [4]:
dataset_root = "https://dprepdata.blob.core.windows.net/demo"

green_path = "/".join([dataset_root, "green-small/*"])
yellow_path = "/".join([dataset_root, "yellow-small/*"])

green_df = dprep.read_csv(path=green_path, header=dprep.PromoteHeadersMode.GROUPED)
# auto_read_file will automatically identify and parse the file type, and is useful if you don't know the file type
yellow_df = dprep.auto_read_file(path=yellow_path)

display(green_df.head(5))
display(yellow_df.head(5))

,VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,...,Extra,MTA_tax,Tip_amount,Tolls_amount,Ehail_fee,Total_amount,Payment_type,Trip_type,Column21,Column22
0,,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,2,2013-08-01 08:14:37,2013-08-01 09:09:06,N,1,0,0,0,0,1,...,0,0,0,0,,21.25,2,,,
2,2,2013-08-01 09:13:00,2013-08-01 11:38:00,N,1,0,0,0,0,2,...,0,0.5,0,0,,75,2,,,
3,2,2013-08-01 09:48:00,2013-08-01 09:49:00,N,5,0,0,0,0,1,...,0.1,0,0,1,,2.1,2,,,
4,2,2013-08-01 10:38:35,2013-08-01 10:38:51,N,1,0,0,0,0,1,...,0,0,0,0,,3.25,2,,,


,vendor_name,Trip_Pickup_DateTime,Trip_Dropoff_DateTime,Passenger_Count,Trip_Distance,Start_Lon,Start_Lat,Rate_Code,store_and_forward,End_Lon,End_Lat,Payment_Type,Fare_Amt,surcharge,mta_tax,Tip_Amt,Tolls_Amt,Total_Amt
0,,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,VTS,2009-01-04 02:52:00,2009-01-04 03:02:00,1,2.6299999999999999,-73.991956999999999,40.721567,,,-73.993803,40.695922000000003,CASH,8.9000000000000004,0.5,,0,0,9.4000000000000004
2,VTS,2009-01-04 03:31:00,2009-01-04 03:38:00,3,4.5499999999999998,-73.982101999999998,40.736289999999997,,,-73.955849999999998,40.768030000000003,Credit,12.1,0.5,,2,0,14.6
3,VTS,2009-01-03 15:43:00,2009-01-03 15:57:00,5,10.35,-74.002587000000005,40.739747999999999,,,-73.869983000000005,40.770225000000003,Credit,23.699999999999999,0,,4.7400000000000002,0,28.440000000000001
4,DDS,2009-01-01 20:52:58,2009-01-01 21:14:00,1,5,-73.974266999999998,40.790954999999997,,,-73.996557999999993,40.731848999999997,CREDIT,14.9,0.5,,3.0499999999999998,0,18.449999999999999


In [5]:
all_columns = dprep.ColumnSelector(term=".*", use_regex=True)
drop_if_all_null = [all_columns, dprep.ColumnRelationship(dprep.ColumnRelationship.ALL)]
useful_columns = [
    "cost", "distance", "dropoff_datetime", "dropoff_latitude", "dropoff_longitude",
    "passengers", "pickup_datetime", "pickup_latitude", "pickup_longitude", "store_forward", "vendor"
]

In [6]:
tmp_df = (green_df
    .replace_na(columns=all_columns)
    .drop_nulls(*drop_if_all_null)
    .rename_columns(column_pairs={
        "VendorID": "vendor",
        "lpep_pickup_datetime": "pickup_datetime",
        "Lpep_dropoff_datetime": "dropoff_datetime",
        "lpep_dropoff_datetime": "dropoff_datetime",
        "Store_and_fwd_flag": "store_forward",
        "store_and_fwd_flag": "store_forward",
        "Pickup_longitude": "pickup_longitude",
        "Pickup_latitude": "pickup_latitude",
        "Dropoff_longitude": "dropoff_longitude",
        "Dropoff_latitude": "dropoff_latitude",
        "Passenger_count": "passengers",
        "Fare_amount": "cost",
        "Trip_distance": "distance"
     })
    .keep_columns(columns=useful_columns))
tmp_df.head(5)

,vendor,pickup_datetime,dropoff_datetime,store_forward,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passengers,distance,cost
0,2,2013-08-01 08:14:37,2013-08-01 09:09:06,N,0,0,0,0,1,.00,21.25
1,2,2013-08-01 09:13:00,2013-08-01 11:38:00,N,0,0,0,0,2,.00,74.5
2,2,2013-08-01 09:48:00,2013-08-01 09:49:00,N,0,0,0,0,1,.00,1
3,2,2013-08-01 10:38:35,2013-08-01 10:38:51,N,0,0,0,0,1,.00,3.25
4,2,2013-08-01 11:51:45,2013-08-01 12:03:52,N,0,0,0,0,1,.00,8.5


In [7]:
green_df = tmp_df

In [8]:
tmp_df = (yellow_df
    .replace_na(columns=all_columns)
    .drop_nulls(*drop_if_all_null)
    .rename_columns(column_pairs={
        "vendor_name": "vendor",
        "VendorID": "vendor",
        "vendor_id": "vendor",
        "Trip_Pickup_DateTime": "pickup_datetime",
        "tpep_pickup_datetime": "pickup_datetime",
        "Trip_Dropoff_DateTime": "dropoff_datetime",
        "tpep_dropoff_datetime": "dropoff_datetime",
        "store_and_forward": "store_forward",
        "store_and_fwd_flag": "store_forward",
        "Start_Lon": "pickup_longitude",
        "Start_Lat": "pickup_latitude",
        "End_Lon": "dropoff_longitude",
        "End_Lat": "dropoff_latitude",
        "Passenger_Count": "passengers",
        "passenger_count": "passengers",
        "Fare_Amt": "cost",
        "fare_amount": "cost",
        "Trip_Distance": "distance",
        "trip_distance": "distance"
    })
    .keep_columns(columns=useful_columns))
tmp_df.head(5)

,vendor,pickup_datetime,dropoff_datetime,passengers,distance,pickup_longitude,pickup_latitude,store_forward,dropoff_longitude,dropoff_latitude,cost
0,VTS,2009-01-04 02:52:00,2009-01-04 03:02:00,1,2.6299999999999999,-73.991956999999999,40.721567,None,-73.993803,40.695922000000003,8.9000000000000004
1,VTS,2009-01-04 03:31:00,2009-01-04 03:38:00,3,4.5499999999999998,-73.982101999999998,40.736289999999997,None,-73.955849999999998,40.768030000000003,12.1
2,VTS,2009-01-03 15:43:00,2009-01-03 15:57:00,5,10.35,-74.002587000000005,40.739747999999999,None,-73.869983000000005,40.770225000000003,23.699999999999999
3,DDS,2009-01-01 20:52:58,2009-01-01 21:14:00,1,5,-73.974266999999998,40.790954999999997,None,-73.996557999999993,40.731848999999997,14.9
4,DDS,2009-01-24 16:18:23,2009-01-24 16:24:56,1,0.40000000000000002,-74.001580000000004,40.719382000000003,None,-74.008377999999993,40.720350000000003,3.7000000000000002


In [9]:
yellow_df = tmp_df
combined_df = green_df.append_rows([yellow_df])

In [10]:
decimal_type = dprep.TypeConverter(data_type=dprep.FieldType.DECIMAL)
combined_df = combined_df.set_column_types(type_conversions={
    "pickup_longitude": decimal_type,
    "pickup_latitude": decimal_type,
    "dropoff_longitude": decimal_type,
    "dropoff_latitude": decimal_type
})
combined_df.keep_columns(columns=[
    "pickup_longitude", "pickup_latitude", 
    "dropoff_longitude", "dropoff_latitude"
]).get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent missing,Error Count,Empty count,0.1% Quantile,1% Quantile,5% Quantile,25% Quantile,50% Quantile,75% Quantile,95% Quantile,99% Quantile,99.9% Quantile,Mean,Standard Deviation,Variance,Skewness,Kurtosis
pickup_longitude,FieldType.DECIMAL,-115.179337,0.000000,7722.0,0.0,7722.0,0.0,0.0,0.0,-88.114046,-73.961840,-73.961964,-73.947693,-73.922097,-73.846670,0.000000,0.000000,0.000000,-68.833579,18.792672,353.164515,3.369970,9.475822
pickup_latitude,FieldType.DECIMAL,0.000000,40.919121,7722.0,0.0,7722.0,0.0,0.0,0.0,0.000000,40.682889,40.675541,40.721075,40.756159,40.803909,40.849406,40.870681,40.891244,37.936742,10.345967,107.039038,-3.391034,9.507615
dropoff_longitude,FieldType.DECIMAL,-115.179337,0.000000,7722.0,0.0,7722.0,0.0,0.0,0.0,-87.699611,-73.984734,-73.985777,-73.956250,-73.928948,-73.866208,0.000000,0.000000,0.000000,-68.896978,18.696526,349.560067,3.393537,9.637675
dropoff_latitude,FieldType.DECIMAL,0.000000,41.008934,7722.0,0.0,7722.0,0.0,0.0,0.0,0.000000,40.662763,40.654851,40.717821,40.756534,40.784688,40.852437,40.879289,40.937291,37.963774,10.290780,105.900157,-3.414918,9.670386


In [11]:
tmp_df = (combined_df
    .drop_nulls(
        columns=["pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude"],
        column_relationship=dprep.ColumnRelationship(dprep.ColumnRelationship.ANY)
    ) 
    .filter(dprep.f_and(
        dprep.col("pickup_longitude") <= -73.72,
        dprep.col("pickup_longitude") >= -74.09,
        dprep.col("pickup_latitude") <= 40.88,
        dprep.col("pickup_latitude") >= 40.53,
        dprep.col("dropoff_longitude") <= -73.72,
        dprep.col("dropoff_longitude") >= -74.09,
        dprep.col("dropoff_latitude") <= 40.88,
        dprep.col("dropoff_latitude") >= 40.53
    )))
tmp_df.keep_columns(columns=[
    "pickup_longitude", "pickup_latitude", 
    "dropoff_longitude", "dropoff_latitude"
]).get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent missing,Error Count,Empty count,0.1% Quantile,1% Quantile,5% Quantile,25% Quantile,50% Quantile,75% Quantile,95% Quantile,99% Quantile,99.9% Quantile,Mean,Standard Deviation,Variance,Skewness,Kurtosis
pickup_longitude,FieldType.DECIMAL,-74.078156,-73.736481,7059.0,0.0,7059.0,0.0,0.0,0.0,-74.076314,-73.962542,-73.962893,-73.948975,-73.927856,-73.866662,-73.830438,-73.823160,-73.769750,-73.913865,0.048711,0.002373,0.402697,-0.613516
pickup_latitude,FieldType.DECIMAL,40.575485,40.879852,7059.0,0.0,7059.0,0.0,0.0,0.0,40.632884,40.713105,40.711600,40.721403,40.758142,40.805145,40.848855,40.867567,40.877690,40.765226,0.048348,0.002338,0.228088,-0.598862
dropoff_longitude,FieldType.DECIMAL,-74.085747,-73.720871,7059.0,0.0,7059.0,0.0,0.0,0.0,-74.078828,-73.985650,-73.985813,-73.959041,-73.936681,-73.884846,-73.815507,-73.776697,-73.733471,-73.920718,0.055961,0.003132,0.648649,0.022914
dropoff_latitude,FieldType.DECIMAL,40.583530,40.879734,7059.0,0.0,7059.0,0.0,0.0,0.0,40.597741,40.695376,40.695115,40.727549,40.758160,40.788378,40.850372,40.867968,40.878586,40.759487,0.050462,0.002546,0.048418,-0.036880


In [12]:
combined_df = tmp_df

In [13]:
combined_df.keep_columns(columns='store_forward').get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent missing,Error Count,Empty count,0.1% Quantile,1% Quantile,5% Quantile,25% Quantile,50% Quantile,75% Quantile,95% Quantile,99% Quantile,99.9% Quantile,Mean,Standard Deviation,Variance,Skewness,Kurtosis
store_forward,FieldType.STRING,N,Y,7059.0,99.0,6960.0,0.014025,0.0,0.0,,,,,,,,,,,,,,


In [14]:
combined_df = combined_df.replace(columns="store_forward", find="0", replace_with="N").fill_nulls("store_forward", "N")

In [15]:
combined_df = combined_df.replace(columns="distance", find=".00", replace_with=0).fill_nulls("distance", 0)
combined_df = combined_df.to_number(["distance"])

In [16]:
tmp_df = (combined_df
    .split_column_by_example(source_column="pickup_datetime")
    .split_column_by_example(source_column="dropoff_datetime"))
tmp_df.head(5)

,vendor,pickup_datetime,pickup_datetime_1,pickup_datetime_2,dropoff_datetime,dropoff_datetime_1,dropoff_datetime_2,store_forward,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passengers,distance,cost
0,2,2013-08-01 17:22:00,2013-08-01,17:22:00,2013-08-01 17:22:00,2013-08-01,17:22:00,N,-73.937767,40.758480,-73.937767,40.758480,1,0.0,2.5
1,2,2013-08-01 17:24:00,2013-08-01,17:24:00,2013-08-01 17:25:00,2013-08-01,17:25:00,N,-73.937927,40.757843,-73.937927,40.757843,1,0.0,2.5
2,2,2013-08-06 06:51:19,2013-08-06,06:51:19,2013-08-06 06:51:36,2013-08-06,06:51:36,N,-73.937721,40.758404,-73.937721,40.758369,1,0.0,3.3
3,2,2013-08-06 13:26:34,2013-08-06,13:26:34,2013-08-06 13:26:57,2013-08-06,13:26:57,N,-73.937691,40.758419,-73.937790,40.758358,1,0.0,3.3
4,2,2013-08-06 13:27:53,2013-08-06,13:27:53,2013-08-06 13:28:08,2013-08-06,13:28:08,N,-73.937805,40.758396,-73.937775,40.758450,1,0.0,3.3


In [17]:
tmp_df_renamed = (tmp_df
    .rename_columns(column_pairs={
        "pickup_datetime_1": "pickup_date",
        "pickup_datetime_2": "pickup_time",
        "dropoff_datetime_1": "dropoff_date",
        "dropoff_datetime_2": "dropoff_time"
    }))
tmp_df_renamed.head(5)

,vendor,pickup_datetime,pickup_date,pickup_time,dropoff_datetime,dropoff_date,dropoff_time,store_forward,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passengers,distance,cost
0,2,2013-08-01 17:22:00,2013-08-01,17:22:00,2013-08-01 17:22:00,2013-08-01,17:22:00,N,-73.937767,40.758480,-73.937767,40.758480,1,0.0,2.5
1,2,2013-08-01 17:24:00,2013-08-01,17:24:00,2013-08-01 17:25:00,2013-08-01,17:25:00,N,-73.937927,40.757843,-73.937927,40.757843,1,0.0,2.5
2,2,2013-08-06 06:51:19,2013-08-06,06:51:19,2013-08-06 06:51:36,2013-08-06,06:51:36,N,-73.937721,40.758404,-73.937721,40.758369,1,0.0,3.3
3,2,2013-08-06 13:26:34,2013-08-06,13:26:34,2013-08-06 13:26:57,2013-08-06,13:26:57,N,-73.937691,40.758419,-73.937790,40.758358,1,0.0,3.3
4,2,2013-08-06 13:27:53,2013-08-06,13:27:53,2013-08-06 13:28:08,2013-08-06,13:28:08,N,-73.937805,40.758396,-73.937775,40.758450,1,0.0,3.3


In [18]:
combined_df = tmp_df_renamed
combined_df.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent missing,Error Count,Empty count,0.1% Quantile,1% Quantile,5% Quantile,25% Quantile,50% Quantile,75% Quantile,95% Quantile,99% Quantile,99.9% Quantile,Mean,Standard Deviation,Variance,Skewness,Kurtosis
vendor,FieldType.STRING,1,VTS,7059.0,0.0,7059.0,0.0,0.0,0.0,,,,,,,,,,,,,,
pickup_datetime,FieldType.STRING,2009-01-01 20:52:58,2013-08-31 23:59:14,7059.0,0.0,7059.0,0.0,0.0,0.0,,,,,,,,,,,,,,
pickup_date,FieldType.STRING,2009-01-01,2013-08-31,7059.0,0.0,7059.0,0.0,0.0,0.0,,,,,,,,,,,,,,
pickup_time,FieldType.STRING,00:00:48,23:59:48,7059.0,0.0,7059.0,0.0,0.0,0.0,,,,,,,,,,,,,,
dropoff_datetime,FieldType.STRING,2009-01-01 21:14:00,2013-09-01 00:25:07,7059.0,0.0,7059.0,0.0,0.0,0.0,,,,,,,,,,,,,,
dropoff_date,FieldType.STRING,2009-01-01,2013-09-01,7059.0,0.0,7059.0,0.0,0.0,0.0,,,,,,,,,,,,,,
dropoff_time,FieldType.STRING,00:00:00,23:59:45,7059.0,0.0,7059.0,0.0,0.0,0.0,,,,,,,,,,,,,,
store_forward,FieldType.STRING,N,Y,7059.0,0.0,7059.0,0.0,0.0,0.0,,,,,,,,,,,,,,
pickup_longitude,FieldType.DECIMAL,-74.0782,-73.7365,7059.0,0.0,7059.0,0.0,0.0,0.0,-74.0763,-73.9625,-73.9629,-73.949,-73.9279,-73.8667,-73.8304,-73.8232,-73.7698,-73.9139,0.0487111,0.00237277,0.402697,-0.613516
pickup_latitude,FieldType.DECIMAL,40.5755,40.8799,7059.0,0.0,7059.0,0.0,0.0,0.0,40.6329,40.7131,40.7116,40.7214,40.7581,40.8051,40.8489,40.8676,40.8777,40.7652,0.0483485,0.00233758,0.228088,-0.598862


# Transform data

In [19]:
tmp_df = (combined_df
    .derive_column_by_example(
        source_columns="pickup_date", 
        new_column_name="pickup_weekday", 
        example_data=[("2009-01-04", "Sunday"), ("2013-08-22", "Thursday")]
    )
    .derive_column_by_example(
        source_columns="dropoff_date",
        new_column_name="dropoff_weekday",
        example_data=[("2013-08-22", "Thursday"), ("2013-11-03", "Sunday")]
    )

    .split_column_by_example(source_column="pickup_time")
    .split_column_by_example(source_column="dropoff_time")
    # the following two split_column_by_example calls reference the generated column names from the above two calls
    .split_column_by_example(source_column="pickup_time_1")
    .split_column_by_example(source_column="dropoff_time_1")
    .drop_columns(columns=[
        "pickup_date", "pickup_time", "dropoff_date", "dropoff_time", 
        "pickup_date_1", "dropoff_date_1", "pickup_time_1", "dropoff_time_1"
    ])

    .rename_columns(column_pairs={
        "pickup_date_2": "pickup_month",
        "pickup_date_3": "pickup_monthday",
        "pickup_time_1_1": "pickup_hour",
        "pickup_time_1_2": "pickup_minute",
        "pickup_time_2": "pickup_second",
        "dropoff_date_2": "dropoff_month",
        "dropoff_date_3": "dropoff_monthday",
        "dropoff_time_1_1": "dropoff_hour",
        "dropoff_time_1_2": "dropoff_minute",
        "dropoff_time_2": "dropoff_second"
    }))

tmp_df.head(5)

,vendor,pickup_datetime,pickup_weekday,pickup_hour,pickup_minute,pickup_second,dropoff_datetime,dropoff_weekday,dropoff_hour,dropoff_minute,dropoff_second,store_forward,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passengers,distance,cost
0,2,2013-08-01 17:22:00,Thursday,17,22,00,2013-08-01 17:22:00,Thursday,17,22,00,N,-73.937767,40.758480,-73.937767,40.758480,1,0.0,2.5
1,2,2013-08-01 17:24:00,Thursday,17,24,00,2013-08-01 17:25:00,Thursday,17,25,00,N,-73.937927,40.757843,-73.937927,40.757843,1,0.0,2.5
2,2,2013-08-06 06:51:19,Tuesday,06,51,19,2013-08-06 06:51:36,Tuesday,06,51,36,N,-73.937721,40.758404,-73.937721,40.758369,1,0.0,3.3
3,2,2013-08-06 13:26:34,Tuesday,13,26,34,2013-08-06 13:26:57,Tuesday,13,26,57,N,-73.937691,40.758419,-73.937790,40.758358,1,0.0,3.3
4,2,2013-08-06 13:27:53,Tuesday,13,27,53,2013-08-06 13:28:08,Tuesday,13,28,08,N,-73.937805,40.758396,-73.937775,40.758450,1,0.0,3.3


In [20]:
tmp_df = tmp_df.drop_columns(columns=["pickup_datetime", "dropoff_datetime"])

In [21]:
type_infer = tmp_df.builders.set_column_types()
type_infer.learn()
type_infer

Column types conversion candidates:
'pickup_weekday': [FieldType.STRING],
'pickup_hour': [FieldType.DECIMAL],
'pickup_longitude': [FieldType.DECIMAL],
'passengers': [FieldType.DECIMAL],
'pickup_minute': [FieldType.DECIMAL],
'store_forward': [FieldType.STRING],
'dropoff_weekday': [FieldType.STRING],
'dropoff_hour': [FieldType.DECIMAL],
'pickup_latitude': [FieldType.DECIMAL],
'distance': [FieldType.DECIMAL],
'pickup_second': [FieldType.DECIMAL],
'dropoff_longitude': [FieldType.DECIMAL],
'cost': [FieldType.DECIMAL],
'dropoff_minute': [FieldType.DECIMAL],
'dropoff_second': [FieldType.DECIMAL],
'vendor': [FieldType.STRING],
'dropoff_latitude': [FieldType.DECIMAL]

In [22]:
tmp_df = type_infer.to_dataflow()
tmp_df.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent missing,Error Count,Empty count,0.1% Quantile,1% Quantile,5% Quantile,25% Quantile,50% Quantile,75% Quantile,95% Quantile,99% Quantile,99.9% Quantile,Mean,Standard Deviation,Variance,Skewness,Kurtosis
vendor,FieldType.STRING,1,VTS,7059.0,0.0,7059.0,0.0,0.0,0.0,,,,,,,,,,,,,,
pickup_weekday,FieldType.STRING,Friday,Wednesday,7059.0,0.0,7059.0,0.0,0.0,0.0,,,,,,,,,,,,,,
pickup_hour,FieldType.DECIMAL,0,23,7059.0,0.0,7059.0,0.0,0.0,0.0,0,3.57523,3,9.91106,15.9327,19,22.0225,23,23,14.2326,6.34926,40.3131,-0.693335,-0.459336
pickup_minute,FieldType.DECIMAL,0,59,7059.0,0.0,7059.0,0.0,0.0,0.0,0,5.32313,4.92308,14.2214,29.5244,44.6436,56.3767,58.9798,59,29.4635,17.4396,304.14,0.00440324,-1.20458
pickup_second,FieldType.DECIMAL,0,59,7059.0,0.0,7059.0,0.0,0.0,0.0,0,4.99286,4.91954,14.6121,29.9239,44.5221,56.6792,59,59,29.6225,17.3868,302.302,-0.0227466,-1.19409
dropoff_weekday,FieldType.STRING,Friday,Wednesday,7059.0,0.0,7059.0,0.0,0.0,0.0,,,,,,,,,,,,,,
dropoff_hour,FieldType.DECIMAL,0,23,7059.0,0.0,7059.0,0.0,0.0,0.0,0,3.23217,2.93333,9.92334,15.9135,19,22.2739,23,23,14.1815,6.45578,41.677,-0.691001,-0.500215
dropoff_minute,FieldType.DECIMAL,0,59,7059.0,0.0,7059.0,0.0,0.0,0.0,0,5.1064,5,14.2051,29.079,44.2937,56.6338,58.9984,59,29.353,17.4241,303.598,0.0142562,-1.21531
dropoff_second,FieldType.DECIMAL,0,59,7059.0,0.0,7059.0,0.0,0.0,0.0,0,5.03373,5,14.7471,29.598,45.3216,56.1044,58.9728,59,29.7923,17.481,305.585,-0.0281313,-1.21965
store_forward,FieldType.STRING,N,Y,7059.0,0.0,7059.0,0.0,0.0,0.0,,,,,,,,,,,,,,


In [23]:
tmp_df = tmp_df.filter(dprep.col("distance") > 0)
tmp_df = tmp_df.filter(dprep.col("cost") > 0)

In [24]:
import os
file_path = os.path.join(os.getcwd(), "dflows.dprep")

dflow_prepared = tmp_df
package = dprep.Package([dflow_prepared])
package.save(file_path)

Package
  name: None
  path: C:\Users\samanoh\Downloads\pytest\dflows.dprep
  dataflows: [
    Dataflow {
      name: dataflow
      steps: 31
    },
  ]

# Train data

In [1]:
import azureml.core
import pandas as pd
from azureml.core.workspace import Workspace
from azureml.train.automl.run import AutoMLRun
import time
import logging
import os

C:\Users\samanoh\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [31]:
ws = Workspace.from_config()
# choose a name for the run history container in the workspace
experiment_name = 'automated-ml-regression'
# project folder
project_folder = './automated-ml-regression'

output = {}
output['SDK version'] = azureml.core.VERSION
#output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data=output, index=['']).T

Found the config file in: C:\Users\samanoh\Downloads\pytest\aml_config\config.json


,
SDK version,1.0.2
Workspace,amlworkspace
Resource Group,ml
Location,eastus
Project Directory,./automated-ml-regression


In [25]:
import azureml.dataprep as dprep

file_path = os.path.join(os.getcwd(), "dflows.dprep")

package_saved = dprep.Package.open(file_path)
dflow_prepared = package_saved.dataflows[0]
dflow_prepared.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent missing,Error Count,Empty count,0.1% Quantile,1% Quantile,5% Quantile,25% Quantile,50% Quantile,75% Quantile,95% Quantile,99% Quantile,99.9% Quantile,Mean,Standard Deviation,Variance,Skewness,Kurtosis
vendor,FieldType.STRING,1,VTS,6148.0,0.0,6148.0,0.0,0.0,0.0,,,,,,,,,,,,,,
pickup_weekday,FieldType.STRING,Friday,Wednesday,6148.0,0.0,6148.0,0.0,0.0,0.0,,,,,,,,,,,,,,
pickup_hour,FieldType.DECIMAL,0,23,6148.0,0.0,6148.0,0.0,0.0,0.0,0,2.90047,2.69355,9.72889,16,19.3713,22.6974,23,23,14.2731,6.59242,43.46,-0.693723,-0.570403
pickup_minute,FieldType.DECIMAL,0,59,6148.0,0.0,6148.0,0.0,0.0,0.0,0,4.99701,4.95833,14.1528,29.3832,44.6825,56.4444,58.9909,59,29.427,17.4333,303.921,0.0120999,-1.20981
pickup_second,FieldType.DECIMAL,0,59,6148.0,0.0,6148.0,0.0,0.0,0.0,0,5.28131,5,14.7832,29.9293,44.725,56.7573,59,59,29.7443,17.3595,301.351,-0.0252399,-1.19616
dropoff_weekday,FieldType.STRING,Friday,Wednesday,6148.0,0.0,6148.0,0.0,0.0,0.0,,,,,,,,,,,,,,
dropoff_hour,FieldType.DECIMAL,0,23,6148.0,0.0,6148.0,0.0,0.0,0.0,0,2.57153,2,9.58795,15.9994,19.6184,22.8317,23,23,14.2105,6.71093,45.0365,-0.687292,-0.61951
dropoff_minute,FieldType.DECIMAL,0,59,6148.0,0.0,6148.0,0.0,0.0,0.0,0,5.44383,4.84694,14.1036,28.8365,44.3102,56.6892,59,59,29.2907,17.4108,303.136,0.0222514,-1.2181
dropoff_second,FieldType.DECIMAL,0,59,6148.0,0.0,6148.0,0.0,0.0,0.0,0,5.07801,5,14.5751,29.5972,45.4649,56.2729,59,59,29.772,17.5337,307.429,-0.0212575,-1.226
store_forward,FieldType.STRING,N,Y,6148.0,0.0,6148.0,0.0,0.0,0.0,,,,,,,,,,,,,,


In [26]:
dflow_X = dflow_prepared.keep_columns(['pickup_weekday','pickup_hour', 'distance','passengers', 'vendor'])
dflow_y = dflow_prepared.keep_columns('cost')

In [27]:
from sklearn.model_selection import train_test_split

x_df = dflow_X.to_pandas_dataframe()
y_df = dflow_y.to_pandas_dataframe()

x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=223)
# flatten y_train to 1d array
y_train.values.flatten()

array([26. ,  6.5, 15.5, ..., 24.5, 11. , 33.5])

# Auto ML

In [28]:
automl_settings = {
    "iteration_timeout_minutes" : 10,
    "iterations" : 30,
    "primary_metric" : 'spearman_correlation',
    "preprocess" : True,
    "verbosity" : logging.INFO,
    "n_cross_validations": 5
}

In [29]:
from azureml.train.automl import AutoMLConfig

# local compute
automated_ml_config = AutoMLConfig(task = 'regression',
                             debug_log = 'automated_ml_errors.log',
                             path = project_folder,
                             X = x_train.values,
                             y = y_train.values.flatten(),
                             **automl_settings)

In [30]:
from azureml.core.experiment import Experiment
experiment=Experiment(ws, experiment_name)
local_run = experiment.submit(automated_ml_config, show_output=True)

Parent Run ID: AutoML_f4581dfe-1623-4e70-b419-4f953f1c9b71
*******************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
*******************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         0   StandardScalerWrapper ExtremeRandomTrees       0:00:28       0.9358    0.9358
         1   MaxAbsScaler ExtremeRandomTrees                0:00:50       0.8347    0.9358
         2   MaxAbsScaler ExtremeRandomTrees                0:00:29       0.9445    0.9445
         3   StandardScalerWrapper GradientBoosting         0:00:28       0.9475    0.9475
         4   StandardScalerWrapper RandomFor

In [32]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'NOTSET', …

In [33]:
children = list(local_run.get_children())
metricslist = {}
for run in children:
    properties = run.get_properties()
    metrics = {k: v for k, v in run.get_metrics().items() if isinstance(v, float)}
    metricslist[int(properties['iteration'])] = metrics

import pandas as pd
rundata = pd.DataFrame(metricslist).sort_index(1)
rundata

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
explained_variance,0.679170,0.128399,0.795674,0.864722,0.757071,0.343023,0.861241,0.576015,0.851909,0.826856,...,0.848275,0.854374,0.809337,0.740873,0.863143,0.836991,0.845184,0.863291,0.857235,NaN
mean_absolute_error,3.147994,6.284894,2.239960,1.554173,2.700708,5.505773,1.609034,4.440999,1.766479,1.845293,...,1.507943,1.747757,1.969119,2.205279,1.634629,1.903894,1.772878,1.597910,1.611230,NaN
median_absolute_error,2.191316,4.157700,1.473957,0.882800,1.827804,4.435721,0.916173,3.624289,0.996712,1.059066,...,0.785500,0.986467,0.985678,0.945032,0.928572,1.258450,1.078046,0.951484,0.924434,NaN
normalized_mean_absolute_error,0.036819,0.073508,0.026198,0.018177,0.031587,0.064395,0.018819,0.051942,0.020661,0.021582,...,0.017637,0.020442,0.023031,0.025793,0.019118,0.022268,0.020735,0.018689,0.018845,NaN
normalized_median_absolute_error,0.025629,0.048628,0.017239,0.010325,0.021378,0.051880,0.010715,0.042389,0.011657,0.012387,...,0.009187,0.011538,0.011528,0.011053,0.010860,0.014719,0.012609,0.011128,0.010812,NaN
normalized_root_mean_squared_error,0.064115,0.111222,0.051103,0.041439,0.055791,0.091583,0.041967,0.073715,0.043420,0.047289,...,0.044447,0.043047,0.049234,0.058158,0.041681,0.045574,0.044393,0.041624,0.042503,NaN
normalized_root_mean_squared_log_error,0.096173,0.164196,0.075845,0.064175,0.090759,0.148363,0.064129,0.125700,NaN,0.064899,...,0.059162,NaN,0.071348,0.066593,0.068683,0.070007,0.066579,0.062397,0.063159,NaN
r2_score,0.678986,0.033656,0.795526,0.864561,0.756788,0.342527,0.861038,0.575564,0.851528,0.823858,...,0.844136,0.854136,0.809155,0.735396,0.862878,0.836810,0.844885,0.863075,0.857014,NaN
root_mean_squared_error,5.481858,9.509450,4.369308,3.543076,4.770093,7.830356,3.588188,6.302657,3.712424,4.043228,...,3.800212,3.680497,4.209513,4.972492,3.563730,3.896581,3.795622,3.558826,3.634029,NaN
root_mean_squared_log_error,0.311204,0.531317,0.245424,0.207661,0.293686,0.480084,0.207515,0.406750,NaN,0.210006,...,0.191440,NaN,0.230872,0.215486,0.222250,0.226534,0.215442,0.201909,0.204376,NaN


In [34]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: automated-ml-regression,
Id: AutoML_f4581dfe-1623-4e70-b419-4f953f1c9b71_14,
Type: None,
Status: Completed)
Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(logger=None, task=None)), ('standardscalerwrapper', <automl.client.core.common.model_wrappers.StandardScalerWrapper object at 0x0000017AE27217B8>), ('elasticnet', ElasticNet(alpha=0.2113157894736842, copy_X=True, fit_intercept=True,
      l1_ratio=0.791578947368421, max_iter=1000, normalize=False,
      positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False))])


# Register model

In [35]:
description = 'Automated Machine Learning Model'
tags = None
local_run.register_model(description=description, tags=tags)
local_run.model_id # Use this id to deploy the model as a web service in Azure

Registering model AutoMLf4581dfe1best


'AutoMLf4581dfe1best'

# Test best accurate model

In [36]:
y_predict = fitted_model.predict(x_test.values)
print(y_predict[:10])

[ 7.4424596  12.27450759 22.94672686  8.00690673  7.06962422 10.51870306
 16.49425585 16.95870828 10.40416495  5.2138921 ]


In [37]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(14, 10))
ax1 = fig.add_subplot(111)

distance_vals = [x[4] for x in x_test.values]
y_actual = y_test.values.flatten().tolist()

ax1.scatter(distance_vals[:100], y_predict[:100], s=18, c='b', marker="s", label='Predicted')
ax1.scatter(distance_vals[:100], y_actual[:100], s=18, c='r', marker="o", label='Actual')

ax1.set_xlabel('distance (mi)')
ax1.set_title('Predicted and Actual Cost/Distance')
ax1.set_ylabel('Cost ($)')

plt.legend(loc='upper left', prop={'size': 12})
plt.rcParams.update({'font.size': 14})
plt.show()

<Figure size 1400x1000 with 1 Axes>